In [1]:
import torch
from torch.autograd import Variable
from torchvision import models
import numpy as np
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import dataset
import time
import random

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
dataset_dir = "../Cat_Dog_data/"
train_path = dataset_dir+"train"
test_path = dataset_dir+"test"

In [4]:
train_data_loader = dataset.loader(train_path)
test_data_loader = dataset.test_loader(test_path)

In [5]:
vgg16 = models.vgg16(pretrained=True)
vgg16.classifier[6] = nn.Linear(4096, 2)
vgg16 = vgg16.to(device)

In [6]:
vgg16.eval()

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [7]:
epoches = 5
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(vgg16.classifier.parameters(), lr=0.0001, momentum=0.9)

In [8]:
def train(model, optimizer, epoches, train_data_loader, test_data_loader, device ):
    
    for i in range(epoches):
        print("Epoch: ", i, end="  ")
        for i, (batch, label) in enumerate(train_data_loader):
            
            batch = batch.to(device)
            label = label.to(device)
            model.zero_grad()
            batch = Variable(batch)
            criterion(model(batch), Variable(label)).backward()
            optimizer.step()
        
        test(model, test_data_loader, device)

In [5]:
def test(model, test_data_loader, device):
    model.eval()
    correct = 0
    total = 0
    for i, (batch, label) in enumerate(test_data_loader):
        batch = batch.to(device)
        output = model(Variable(batch))
        pred = output.data.max(1)[1]
        correct += pred.cpu().eq(label).sum()
        total += label.size(0)

    print("Accuracy :", float(correct) / total)

        

In [10]:
start = time.time()
train(vgg16, optimizer, epoches, train_data_loader, test_data_loader, device)
end = time.time()
print(end - start)

Epoch:  0  Accuracy : 0.9824
Epoch:  1  Accuracy : 0.9844
Epoch:  2  Accuracy : 0.9872
Epoch:  3  Accuracy : 0.9876
Epoch:  4  Accuracy : 0.9872
11507.489993333817


In [11]:
torch.save(vgg16, "vgg16")

In [8]:
vgg16 = torch.load("vgg16")

In [10]:
test(vgg16, test_data_loader, device)

Accuracy : 0.9872


In [11]:
total_params = sum(p.numel() for p in vgg16.parameters())
print("Numbre of parameters:", total_params)

Numbre of parameters: 134268738
